In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import StratifiedKFold

In [ ]:
import lightgbm
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [ ]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise  import cosine_similarity
from sklearn.metrics.pairwise import chi2_kernel

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization

In [ ]:
ss = StandardScaler()
ss2 = StandardScaler()

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv', sep=',')

sub_sample = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv', sep=',')

In [ ]:
test= pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv', sep=',')

In [ ]:
print(train.shape, test.shape, sub_sample.shape)

In [ ]:
train = train.set_index('id')

In [ ]:
l=[]
for i in train.columns:
    if train[i].dtype!='O':
        l.append(i)

In [ ]:
l2=[]
for i in train.columns:
    if train[i].dtype=='O':
        l2.append(i)
l2.append('target')

In [ ]:
train[l].describe()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data=train[l[0:-1]])

In [ ]:
train['target'].value_counts()

In [ ]:
train['target'].value_counts(normalize=True)

In [ ]:
df=train[l]
df0= df[df['target']==0]
df1= df[df['target']==1]

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(data=train[l[1:]].corr())

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10,10))
#plt.figure(figsize=(10,10))
sns.boxplot(ax=axes[0],data=train[l[:-1]])
sns.boxplot(ax=axes[1],data=df0[l[:-1]])
sns.boxplot(ax=axes[2],data=df1[l[:-1]])

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(data=df0[l[:]].corr())

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(data=df1[l[:]].corr())

In [ ]:
train[l[:-1]].isna().sum()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10,10))
#plt.figure(figsize=(10,10))
sns.boxplot(ax=axes[0],data=(train[l[:-1]])**2)
sns.boxplot(ax=axes[1],data=(df0[l[:-1]])**2)
sns.boxplot(ax=axes[2],data=(df1[l[:-1]])**2)

In [ ]:
train[l2[:-1]].describe()

In [ ]:
df_cat = train[l2[:-1]]
df_cat2 = train[l2]
df_num=train[l[:-1]]
df_target= train[l[-1]]

In [ ]:
df_cat.head()

In [ ]:
#le = LabelEncoder()

In [ ]:
#df_cat3=df_cat.apply(le.fit_transform)

In [ ]:
#df_cat = df_cat.astype('category')

In [ ]:
import category_encoders as ce
woe=ce.woe.WOEEncoder(return_df=True, drop_invariant=True, handle_missing='value')
woe_enc=woe.fit_transform(df_cat, df_target)
woe_enc= woe_enc.set_index(df_cat.index)
#woe_enc = woe_enc.astype('category')

In [ ]:
woe_enc.head()

In [ ]:
te=ce.target_encoder.TargetEncoder(return_df=True, drop_invariant=True, handle_missing='value', handle_unknown='value', smoothing=0.2)
te_enc=te.fit_transform(df_cat, df_target)
te_enc= te_enc.set_index(df_cat.index)

In [ ]:
te_enc.columns = ['tecat0', 'tecat1', 'tecat2', 'tecat3', 'tecat4', 'tecat5', 'tecat6', 'tecat7', 'tecat8',
       'tecat9', 'tecat10', 'tecat11', 'tecat12', 'tecat13', 'tecat14', 'tecat15', 'tecat16',
       'tecat17', 'tecat18']

In [ ]:
#hashe = ce.hashing.HashingEncoder(return_df=True, drop_invariant=True)
#hash_enc=hashe.fit_transform(df_cat, df_target)
#hash_enc= hash_enc.set_index(df_cat.index)

In [ ]:
#hash_enc.head()

In [ ]:
df= pd.concat([woe_enc, te_enc, df_num], axis=1)

In [ ]:
km = KMeans(n_clusters=4, random_state=15)
km_df = pd.DataFrame(km.fit_predict(df), index=df.index, columns=['cl'])
km_df = km_df.astype('str')
km_df = pd.get_dummies(km_df)
km_df = km_df.astype('int')

In [ ]:
km_df.head()

In [ ]:
df= pd.concat([df, km_df], axis=1)

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
#df = ss.fit_transform(df)
#df = pd.DataFrame(df, index=train.index, columns=train.columns[:-1])

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df1 = df[['cat0',    'cat1',    'cat2',    'cat3',    'cat4',    'cat5',
          'cat6',    'cat7',    'cat8',    'cat9',   'cat10',   'cat11',
         'cat12',   'cat13',   'cat14',   'cat15',   'cat16',   'cat17',
         'cat18', 'cont0',   'cont1',
         'cont2',   'cont3',   'cont4',   'cont5',   'cont6',   'cont7',
         'cont8',   'cont9',  'cont10','cl_0', 'cl_1', 'cl_2',
       'cl_3']]

In [ ]:
df2 = df[['tecat0', 'tecat1', 'tecat2', 'tecat3', 'tecat4',
       'tecat5', 'tecat6', 'tecat7', 'tecat8', 'tecat9', 'tecat10', 'tecat11',
       'tecat12', 'tecat13', 'tecat14', 'tecat15', 'tecat16', 'tecat17',
       'tecat18','cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
       'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cl_0', 'cl_1', 'cl_2',
       'cl_3']]

### Autoencoder

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df1, df_target, test_size=0.30, random_state=1)
# scale data
t = MinMaxScaler()
t.fit(X_train)
X_train = t.transform(X_train)
X_test = t.transform(X_test)

In [ ]:
n_inputs =df1.shape[1]
n_inputs

In [ ]:
# define encoder
visible = Input(shape=(n_inputs,))
# encoder level 1
e = Dense(n_inputs*3)(visible)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
# encoder level 2
e = Dense(n_inputs)(e)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
# bottleneck
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(e)
# define decoder, level 1
d = Dense(n_inputs)(bottleneck)
d = BatchNormalization()(d)
d = LeakyReLU()(d)
# decoder level 2
d = Dense(n_inputs*3)(d)
d = BatchNormalization()(d)
d = LeakyReLU()(d)
# output layer
output = Dense(n_inputs, activation='linear')(d)

In [ ]:
# define autoencoder model
#model = Model(inputs=visible, outputs=output)
# compile autoencoder model
#model.compile(optimizer='adam', loss='mse')

In [ ]:
# plot the autoencoder
#plot_model(model, 'autoencoder_no_compress.png', show_shapes=True)
# fit the autoencoder model to reconstruct input
#history = model.fit(X_train, X_train, epochs=100, batch_size=16, verbose=2, validation_data=(X_test,X_test))

In [ ]:
# plot loss
#plt.plot(history.history['loss'], label='train')
#plt.plot(history.history['val_loss'], label='test')
#plt.legend()
#plt.show()
# define an encoder model (without the decoder)
#encoder = Model(inputs=visible, outputs=bottleneck)
#plot_model(encoder, 'encoder_no_compress.png', show_shapes=True)
# save the encoder to file
#encoder.save('encoder.h5')

In [ ]:
#df_encode = encoder.predict(df1)

In [ ]:
#df_encode = pd.DataFrame(df_encode,index=df1.index)

In [ ]:
#df_encode.head()

In [ ]:
#df_plus = pd.concat([df1, df_encode], axis=1)

## Optuna

In [ ]:
import optuna

In [ ]:
def objective(trial , data = df1 , target = df_target):
    train_x , test_x , train_y , test_y = train_test_split(data , target , \
            test_size = 0.028059109276941666 , random_state = 42)

    #test_size = 0.028059109276941666
    params = {
        'reg_alpha' : trial.suggest_loguniform('reg_alpha' , 1e-5 , 10),
        'reg_lambda' : trial.suggest_loguniform('reg_lambda' , 1e-5 , 10),
        'num_leaves' : trial.suggest_int('num_leaves' , 11 , 800),
        'learning_rate' : trial.suggest_uniform('learning_rate' , 0.0000001 , 0.1),
        'max_depth' : trial.suggest_int('max_depth' , 5 , 400),
        'n_estimators' : trial.suggest_int('n_estimators' , 1 , 9999),
        'min_child_samples' : trial.suggest_int('min_child_samples' , 1 , 110),
        'min_child_weight' : trial.suggest_loguniform('min_child_weight' , 1e-5 , 1),
        'subsample' : trial.suggest_uniform('subsample' , 1e-5 , 1.0),
        'colsample_bytree' : trial.suggest_loguniform('colsample_bytree' , 1e-5 , 1),
        'random_state' : trial.suggest_categorical('random_state' , [1,42,2021,555]),
        'metric' : 'auc',
        'device_type' : 'cpu',
    }
    model = lightgbm.LGBMClassifier(**params)
    model.fit(train_x , train_y , eval_set = [(test_x , test_y)] , early_stopping_rounds = 200 , \
             verbose = False)
    preds = model.predict_proba(test_x)[:,1]
    auc = roc_auc_score(test_y , preds)
    return auc

In [ ]:
#study = optuna.create_study(direction = 'maximize' , study_name = 'lgbm')
#study.optimize(objective , n_trials = 1)
#print('numbers of the finished trials:' , len(study.trials))
#print('the best params:' , study.best_trial.params)
#print('the best value:' , study.best_value)

In [ ]:
#Parametri con df1
#the best value: 0.8990729898631389
parameters_df11={'reg_alpha': 4.2454113913576315, 'reg_lambda': 0.8899994822202811, 'num_leaves': 189, 'learning_rate': 0.025556695055007152, 'max_depth': 88, 'n_estimators': 5023, 'min_child_samples': 84, 'min_child_weight': 0.00039809767984828675, 'subsample': 0.48961516469429756, 'colsample_bytree': 0.1393679695029006, 'random_state': 555}
# parametri con piu' regularization Trial 23 finished with value: 0.8977186777333175
parameters_df12= {'reg_alpha': 0.12852445867029427, 'reg_lambda': 8.661870320090047, 'num_leaves': 126, 'learning_rate': 0.026729106092421163, 'max_depth': 145, 'n_estimators': 2840, 'min_child_samples': 100, 'min_child_weight': 2.8395788296418357e-05, 'subsample': 0.4763268318189475, 'colsample_bytree': 0.16793586245071554, 'random_state': 2021}
#the best value: 0.8986825515857123
parameters_df13= {'reg_alpha': 0.01700126780654701, 'reg_lambda': 0.0027814146390216688, 'num_leaves': 238, 'learning_rate': 0.021938334019234108, 'max_depth': 54, 'n_estimators': 6397, 'min_child_samples': 50, 'min_child_weight': 0.8400282536027239, 'subsample': 0.47915115184384754, 'colsample_bytree': 0.3278563707693388, 'random_state': 42}

In [ ]:
#Parametri con df2
#con piu' regularization Trial 36 finished with value: 0.8978443461542528
parameters_df22={'reg_alpha': 9.118907428042913, 'reg_lambda': 0.12959968423610096, 'num_leaves': 17, 'learning_rate': 0.0776102432033011, 'max_depth': 17, 'n_estimators': 3686, 'min_child_samples': 86, 'min_child_weight': 0.0011355918176110865, 'subsample': 0.8367994419441532, 'colsample_bytree': 0.4836335537092214, 'random_state': 42}
#the best value: 0.8982749366666394
parameters_df21= {'reg_alpha': 0.537327486805204, 'reg_lambda': 0.019584599849890916, 'num_leaves': 82, 'learning_rate': 0.05218198433646826, 'max_depth': 71, 'n_estimators': 4340, 'min_child_samples': 62, 'min_child_weight': 0.0009480351949410995, 'subsample': 0.6752872238604577, 'colsample_bytree': 0.16192808258013053, 'random_state': 42}
#the best value: 0.8991129218847445
parameters_df23 = {'reg_alpha': 0.0001543251818272485, 'reg_lambda': 0.014373071877816938, 'num_leaves': 408, 'learning_rate': 0.020002205506641287, 'max_depth': 123, 'n_estimators': 2391, 'min_child_samples': 99, 'min_child_weight': 0.0048616493226546756, 'subsample': 0.2549660532449299, 'colsample_bytree': 0.21391734300955667, 'random_state': 555}

In [ ]:
param1={'reg_alpha': 3.1537282825971684, 'reg_lambda': 0.0006331603037539777, 'num_leaves': 210, 'learning_rate': 0.030542905754683533, 'max_depth': 46, 'n_estimators': 9728, 'min_child_samples': 3, 'min_child_weight': 0.004536270550575023, 'subsample': 0.5957073001294086, 'colsample_bytree': 0.516835733282878, 'random_state': 42}

In [ ]:
param2={'reg_alpha': 3.4050634203247796, 'reg_lambda': 8.151186640988037, 'num_leaves': 170, 'learning_rate': 0.02144578398400834, 'max_depth': 38, 'n_estimators': 1055, 'min_child_samples': 13, 'min_child_weight': 0.0115577715775169, 'subsample': 0.6052880249006449, 'colsample_bytree': 0.3496802480226771, 'random_state': 42}

In [ ]:
param3={'reg_alpha': 9.786060648128762, 'reg_lambda': 0.0006948954569416347, 'num_leaves': 137, 'learning_rate': 0.023424207089337176, 'max_depth': 64, 'n_estimators': 2583, 'min_child_samples': 1, 'min_child_weight': 1.862399743644229e-05, 'subsample': 0.24617098080102506, 'colsample_bytree': 0.17174339489842147, 'random_state': 42}

In [ ]:
param4={'reg_alpha': 3.98196662753181, 'reg_lambda': 2.758490252875887e-05, 'num_leaves': 133, 'learning_rate': 0.02661593085308742, 'max_depth': 79, 'n_estimators': 5569, 'min_child_samples': 54, 'min_child_weight': 3.758582497733011e-05, 'subsample': 0.2685866648258971, 'colsample_bytree': 0.225831030612319, 'random_state': 42}

with target encoding

In [ ]:
#0.8856023921629884
param_plus =  {'reg_alpha': 0.9743822716453967, 'reg_lambda': 1.8262870443117842, 'num_leaves': 138, 'learning_rate': 0.06978622282435892, 'max_depth': 372, 'n_estimators': 4167, 'min_child_samples': 101, 'min_child_weight': 0.0017234645371239574, 'subsample': 0.6433908875083164, 'colsample_bytree': 0.13564310243636946, 'random_state': 1}

In [ ]:
params1= {'reg_alpha': 6.7779388151102395, 'reg_lambda': 0.2242114494396302, 'num_leaves': 282, 'learning_rate': 0.003404453895520132, 'max_depth': 52, 'n_estimators': 6556, 'min_child_samples': 50, 'min_child_weight': 0.00024094623793528765, 'subsample': 0.1272898098486071, 'colsample_bytree': 0.16134140354767543, 'random_state': 2021}

In [ ]:
#Trial 40 finished with value: 0.8987909699605494 
params2= {'reg_alpha': 0.4171907072665806, 'reg_lambda': 0.010276748360125343, 'num_leaves': 300, 'learning_rate': 0.00771068552120896, 'max_depth': 62, 'n_estimators': 6554, 'min_child_samples': 26, 'min_child_weight': 0.007938177310401738, 'subsample': 0.12193386210016402, 'colsample_bytree': 0.14061264083550198, 'random_state': 2021}

In [ ]:
params3={'reg_alpha': 3.57600648867944, 'reg_lambda': 0.19373229864787594, 'num_leaves': 218, 'learning_rate': 0.08603855902510578, 'max_depth': 14, 'n_estimators': 6676, 'min_child_samples': 62, 'min_child_weight': 0.00937274804904474, 'subsample': 0.6639257272064, 'colsample_bytree': 0.24648367545438585, 'random_state': 1}

# TEST

In [ ]:
test = test.set_index('id')

In [ ]:
lt=[]
for i in test.columns:
    if test[i].dtype!='O':
        lt.append(i)

In [ ]:
lt2=[]
for i in test.columns:
    if test[i].dtype=='O':
        lt2.append(i)

In [ ]:
dft_cat = test[lt2]
dft_cat2 = test[lt2]
dft_num=test[lt]

In [ ]:
woet_enc=woe.transform(dft_cat)
woet_enc= woet_enc.set_index(dft_cat.index)

In [ ]:
tet_enc=te.transform(dft_cat)
tet_enc= tet_enc.set_index(dft_cat.index)

In [ ]:
tet_enc.columns = ['tecat0', 'tecat1', 'tecat2', 'tecat3', 'tecat4', 'tecat5', 'tecat6', 'tecat7', 'tecat8',
       'tecat9', 'tecat10', 'tecat11', 'tecat12', 'tecat13', 'tecat14', 'tecat15', 'tecat16',
       'tecat17', 'tecat18']

In [ ]:
dft= pd.concat([woet_enc, tet_enc, dft_num], axis=1)

In [ ]:
km_dft = pd.DataFrame(km.predict(dft), index=dft.index, columns=['cl'])
km_dft = km_dft.astype('str')
km_dft = pd.get_dummies(km_dft)
km_dft = km_dft.astype('int')

In [ ]:
dft= pd.concat([dft, km_dft], axis=1)

In [ ]:
dft1 = dft[['cat0',    'cat1',    'cat2',    'cat3',    'cat4',    'cat5',
          'cat6',    'cat7',    'cat8',    'cat9',   'cat10',   'cat11',
         'cat12',   'cat13',   'cat14',   'cat15',   'cat16',   'cat17',
         'cat18', 'cont0',   'cont1', 'cont2',   'cont3',   'cont4',   'cont5',   'cont6',   'cont7',
         'cont8',   'cont9',  'cont10','cl_0', 'cl_1', 'cl_2', 'cl_3']]

In [ ]:
dft2 = dft[['tecat0', 'tecat1', 'tecat2', 'tecat3', 'tecat4',
       'tecat5', 'tecat6', 'tecat7', 'tecat8', 'tecat9', 'tecat10', 'tecat11',
       'tecat12', 'tecat13', 'tecat14', 'tecat15', 'tecat16', 'tecat17',
       'tecat18','cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
       'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cl_0', 'cl_1', 'cl_2',
       'cl_3']]

In [ ]:
df1.shape

In [ ]:
dft1.shape

In [ ]:
df_target.shape

In [ ]:
#dft_encode = encoder.predict(dft1)

In [ ]:
#dft_encode = pd.DataFrame(dft_encode, index=dft.index)

In [ ]:
#dft_plus = pd.concat([dft1, dft_encode], axis=1)

In [ ]:
#dft_plus.shape

In [ ]:
#df_plus.shape

param_plus['metric'] = 'auc'
param_plus['device'] = 'cpu'
preds5 = np.zeros(dft_plus.shape[0])
oof_preds5 = np.zeros(df_plus.shape[0])
kf = StratifiedKFold(n_splits = 10 , random_state = 42 , shuffle = True)
roc5 = []
n = 0
for trn_idx , val_idx in kf.split(df_plus , df_target):
    train_x = df_plus.iloc[trn_idx]
    train_y = df_target.iloc[trn_idx]
    val_x = df_plus.iloc[val_idx]
    val_y = df_target.iloc[val_idx]
    
    model5 = lightgbm.LGBMClassifier(**param_plus)
    model5.fit(train_x , train_y , eval_set = [(val_x , val_y)] , early_stopping_rounds = 200 , \
             verbose = False)
    preds5 += model5.predict_proba(dft_plus)[:,1]/kf.n_splits
    oof_preds5 += model5.predict_proba(df_plus)[:,1]/kf.n_splits
    roc5.append(roc_auc_score(val_y , model5.predict_proba(val_x)[:,1]))
    print(n+1 , roc5[n])
    n+=1

In [ ]:
parameters_df11['metric'] = 'auc'
parameters_df11['device'] = 'cpu'
preds = np.zeros(dft1.shape[0])
oof_preds = np.zeros(df1.shape[0])
kf = StratifiedKFold(n_splits = 10 , random_state = 22 , shuffle = True)
roc = []
n = 0
for trn_idx , val_idx in kf.split(df1 , df_target):
    train_x = df1.iloc[trn_idx]
    train_y = df_target.iloc[trn_idx]
    val_x = df1.iloc[val_idx]
    val_y = df_target.iloc[val_idx]
    
    model = lightgbm.LGBMClassifier(**parameters_df11)
    model.fit(train_x , train_y , eval_set = [(val_x , val_y)] , early_stopping_rounds = 4000 , \
             verbose = False)
    clf = CalibratedClassifierCV(model, cv='prefit', method='sigmoid')
    clf.fit(train_x , train_y)
    preds += clf.predict_proba(dft1)[:,1]/kf.n_splits
    oof_preds += clf.predict_proba(df1)[:,1]/kf.n_splits
    roc.append(roc_auc_score(val_y , clf.predict_proba(val_x)[:,1]))
    print(n+1 , roc[n])
    n+=1

parameters_df13['metric'] = 'auc'
parameters_df13['device'] = 'cpu'
preds2 = np.zeros(dft1.shape[0])
oof_preds2 = np.zeros(df1.shape[0])
kf = StratifiedKFold(n_splits = 10 , random_state = 42 , shuffle = True)
roc2 = []
n = 0
for trn_idx , val_idx in kf.split(df1 , df_target):
    train_x = df1.iloc[trn_idx]
    train_y = df_target.iloc[trn_idx]
    val_x = df1.iloc[val_idx]
    val_y = df_target.iloc[val_idx]
    
    model2 = lightgbm.LGBMClassifier(**parameters_df13)
    model2.fit(train_x , train_y , eval_set = [(val_x , val_y)] , early_stopping_rounds = 200 , \
             verbose = False)
    preds2 += model2.predict_proba(dft1)[:,1]/kf.n_splits
    oof_preds2 += model2.predict_proba(df1)[:,1]/kf.n_splits
    roc2.append(roc_auc_score(val_y , model2.predict_proba(val_x)[:,1]))
    print(n+1 , roc2[n])
    n+=1

In [ ]:
parameters_df21['metric'] = 'auc'
parameters_df21['device'] = 'cpu'
preds3 = np.zeros(dft2.shape[0])
oof_preds3 = np.zeros(df2.shape[0])
kf = StratifiedKFold(n_splits = 10 , random_state = 22 , shuffle = True)
roc3 = []
n = 0
for trn_idx , val_idx in kf.split(df2, df_target):
    train_x = df2.iloc[trn_idx]
    train_y = df_target.iloc[trn_idx]
    val_x = df2.iloc[val_idx]
    val_y = df_target.iloc[val_idx]
    
    model3 = lightgbm.LGBMClassifier(**parameters_df21)
    model3.fit(train_x , train_y , eval_set = [(val_x , val_y)] , early_stopping_rounds = 4000 , \
             verbose = False)
    clf3 = CalibratedClassifierCV(model3, cv='prefit', method='sigmoid')
    clf3.fit(train_x , train_y)
    preds3 += clf3.predict_proba(dft2)[:,1]/kf.n_splits
    oof_preds3 += clf3.predict_proba(df2)[:,1]/kf.n_splits
    roc3.append(roc_auc_score(val_y , clf3.predict_proba(val_x)[:,1]))
    print(n+1 , roc3[n])
    n+=1

parameters_df23['metric'] = 'auc'
parameters_df23['device'] = 'cpu'
preds4 = np.zeros(dft2.shape[0])
oof_preds4 = np.zeros(df2.shape[0])
kf = StratifiedKFold(n_splits = 10 , random_state = 42 , shuffle = True)
roc4 = []
n = 0
for trn_idx , val_idx in kf.split(df , df_target):
    train_x = df2.iloc[trn_idx]
    train_y = df_target.iloc[trn_idx]
    val_x = df2.iloc[val_idx]
    val_y = df_target.iloc[val_idx]
    
    model4 = lightgbm.LGBMClassifier(**parameters_df23)
    model4.fit(train_x , train_y , eval_set = [(val_x , val_y)] , early_stopping_rounds = 200 , \
             verbose = False)
    preds4 += model4.predict_proba(dft2)[:,1]/kf.n_splits
    oof_preds4 += model4.predict_proba(df2)[:,1]/kf.n_splits
    roc4.append(roc_auc_score(val_y , model4.predict_proba(val_x)[:,1]))
    print(n+1 , roc4[n])
    n+=1

In [ ]:
sub_sample.head()

In [ ]:
pro= (preds*0.9)+(preds3*0.1)

In [ ]:
sub_sample['target'] = pro

In [ ]:
sub_sample.head()

In [ ]:
sub_sample.to_csv('submission.csv',index=False)